In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import numpy as np
# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Trouble_condition_Query='''

select dealer_name,customer_id,unit_id,device_id,tcon_id,trouble_condition_name,trouble_condition_group_id,trouble_condition_group_desc, start_date_utc,end_date_utc,trouble_condition_start_ind,trouble_condition_closed_ind,ncid
from `cio-datahub-enterprise-pr-183a.src_adc.bq_troublecondition_data`
where dealer_name='TELUS Communications Inc.'

'''

In [ ]:
Trouble_condition_DF= extract_bq_data(bq_client,sql=Trouble_condition_Query)

In [ ]:
Trouble_condition_DF.info()

In [ ]:
Trouble_condition_DF['start_date_utc'].max(),Trouble_condition_DF['start_date_utc'].min()

In [ ]:
Trouble_condition_DF['start_date_utc'].value_counts().sort_values()

In [ ]:
Trouble_condition_DF.head()

In [ ]:
Trouble_condition_DF['trouble_condition_name'].value_counts(normalize=True)*100

In [ ]:
Trouble_condition_DF['trouble_condition_name'].value_counts()

In [ ]:
TC_list = ['CameraNotReachable','CameraNotCommunicating','BroadbandCommFailure']

TC_Camera_issue = Trouble_condition_DF[Trouble_condition_DF['trouble_condition_name'].isin(TC_list)]


In [ ]:
TC_Camera_issue.info()

In [ ]:
TC_Camera_issue['customer_id'].nunique()

In [ ]:
Camera_Query='''

select customer_id,unit_id,device_id,model
from `cio-datahub-enterprise-pr-183a.src_adc.bq_camera`
group by customer_id,unit_id,device_id,model
order by customer_id,unit_id,device_id,model

'''

In [ ]:
Camera_DF= extract_bq_data(bq_client,sql=Camera_Query)

In [ ]:
Camera_DF.info()

In [ ]:
Camera_DF.head()

In [ ]:
Camera_DF.drop_duplicates(keep=False, inplace=True)


In [ ]:
Camera_DF.info()

In [ ]:
Camera_DF['Camera_flag']=1

In [ ]:
Merge_Data=Trouble_condition_DF.merge(Camera_DF, on=['customer_id','unit_id','device_id'], how='left')

In [ ]:
Merge_Data.info()

In [ ]:
Merge_Data['Camera_flag'].isna().sum()*100/Merge_Data.shape[0]

In [ ]:
Merge_Data.head()

In [ ]:
Merge_Data_with_Camera=Merge_Data[Merge_Data['Camera_flag']==1]

In [ ]:
Merge_Data_with_Camera['trouble_condition_name'].value_counts(normalize=True)*100

In [ ]:
Merge_Data_with_Camera.info()

In [ ]:
Merge_Data_with_Camera['customer_id'].nunique()

In [ ]:
len(Merge_Data_with_Camera['customer_id'].unique())

In [ ]:
Merge_Data_with_Camera['trouble_condition_group_desc'].value_counts(normalize=True)*100

In [ ]:
Customer_Details_Query='''

SELECT customer_id,dealer_customer_id,primary_login_id,dealer_name,join_date,account_type_name,customer_type_name,primary_email,primary_phone,last_updt_ts
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id


'''

In [ ]:
Query_Termination_details='''

select customer_id,term_date,term_reason,1 as Churn_flag from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_terminated_details`
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1



'''

In [ ]:
Customer_Details_DF=extract_bq_data(bq_client,sql=Customer_Details_Query)

In [ ]:
Termination_details=extract_bq_data(bq_client,sql=Query_Termination_details)

In [ ]:
Customer_base=Customer_Details_DF.merge(Termination_details,on='customer_id',how='left')

In [ ]:
Customer_base.head()

In [ ]:
Customer_base['Churn_flag'].fillna(0,inplace=True)

In [ ]:
Customer_base['Churn_flag'].value_counts(normalize=True)*100

In [ ]:
Customer_base.info()

In [ ]:
Merge_Data_with_Camera_1=Merge_Data_with_Camera.merge(Customer_base,on='customer_id',how='left')

In [ ]:
Merge_Data_with_Camera_1.info()

In [ ]:
Merge_Data_with_Camera_1.head()

In [ ]:
Merge_Data_with_Camera_1['dealer_name_x'].value_counts()

In [ ]:
Merge_Data_with_Camera_1['Churn_flag'].isna().sum()*100/Merge_Data_with_Camera_1.shape[0]

In [ ]:
Merge_Data_with_Camera_1['Churn_flag'].value_counts(normalize=True)*100

In [ ]:
Merge_Data_with_Camera_1['customer_id'].nunique()

In [ ]:
Merge_Data_with_Camera_1['model'].isna().sum()

In [ ]:
pd.DataFrame(Merge_Data_with_Camera_1.groupby(['model']).agg(
    Total_incidents=('customer_id','count'), 
    Unique_Customers=('customer_id','nunique'), 
    Churn_counts = ('Churn_flag','sum'),
     Churn_rate = ('Churn_flag',lambda x: x.mean()*100),
     ).reset_index().sort_values('Unique_Customers',ascending=False))


In [ ]:
Merge_Data_with_Camera_model=Merge_Data_with_Camera_1[['customer_id','model','Churn_flag']]

In [ ]:
Merge_Data_with_Camera_model.drop_duplicates(inplace=True,keep=False)

In [ ]:
Merge_Data_with_Camera_model['customer_id'].nunique()

In [ ]:
Merge_Data_with_Camera_model['customer_id'].value_counts()

In [ ]:
pd.DataFrame(Merge_Data_with_Camera_model.groupby(['model']).agg(
    Total_incidents=('customer_id','count'), 
    Unique_Customers=('customer_id','nunique'), 
    Churn_counts = ('Churn_flag','sum'),
     Churn_rate = ('Churn_flag',lambda x: x.mean()*100),
     ).reset_index().sort_values('Unique_Customers',ascending=False))


In [ ]:
pd.DataFrame(Merge_Data_with_Camera_1.groupby(['trouble_condition_name']).agg(
    Total_incidents=('customer_id','count'), 
    Unique_Customers=('customer_id','nunique'), 
    Churn_counts = ('Churn_flag','sum'),
     Churn_rate = ('Churn_flag',lambda x: x.mean()*100),
     ).reset_index().sort_values('Unique_Customers',ascending=False))

In [ ]:
Merge_Data_with_TC_cond=Merge_Data_with_Camera_1[['customer_id','trouble_condition_name','Churn_flag']]

In [ ]:
Merge_Data_with_TC_cond.info()

In [ ]:
Merge_Data_with_TC_cond['trouble_condition_name'].isna().sum()

In [ ]:
Merge_Data_with_TC_cond.drop_duplicates(inplace=True,keep=False)

In [ ]:
Merge_Data_with_TC_cond.info()

In [ ]:
pd.DataFrame(Merge_Data_with_TC_cond.groupby(['trouble_condition_name']).agg(
    Total_incidents=('customer_id','count'), 
    Unique_Customers=('customer_id','nunique'), 
    Churn_counts = ('Churn_flag','sum'),
     Churn_rate = ('Churn_flag',lambda x: x.mean()*100),
     ).reset_index().sort_values('Unique_Customers',ascending=False))

In [ ]:
Merge_Data_with_Camera_1.value_counts('trouble_condition_name')

In [ ]:
Merge_Data_with_Camera_1.groupby(['trouble_condition_name']).agg(
     Churn_counts = ('Churn_flag','sum'),
     Churn_rate = ('Churn_flag',lambda x: x.mean()*100),
     ).reset_index().sort_values('Churn_counts',ascending=False)

In [ ]:
Customer_level_data= Merge_Data_with_Camera_1[['customer_id','dealer_customer_id','dealer_name_x','trouble_condition_name','trouble_condition_group_desc','account_type_name','customer_type_name','join_date','term_date','term_reason','primary_phone','Churn_flag']].drop_duplicates(keep=False, inplace=False)

In [ ]:
Customer_level_data.info()

In [ ]:
Customer_level_data['customer_id'].nunique()

In [ ]:
Camera_issue_DF.info()

In [ ]:
Camera_issue_DF.head()

In [ ]:
Camera_issue_DF.groupby('trouble_condition_name')['Churn_flag'].mean()*100

In [ ]:
# Camera_issue_DF['customer_id'].value_counts()

In [ ]:
Camera_issue_Customer_level= Merge_Data[['customer_id','dealer_customer_id','dealer_name_x','account_type_name','customer_type_name','join_date','term_date','term_reason','primary_phone','Churn_flag']].drop_duplicates(keep=False, inplace=False)

In [ ]:
# Camera_issue_Customer_level['customer_id'].value_counts()

In [ ]:
Camera_issue_Customer_level.info()

In [ ]:
Camera_issue_Customer_level['dealer_name_x'].value_counts()

In [ ]:
Camera_issue_Customer_level['Churn_flag'].value_counts(normalize=True)*100

In [ ]:
Telus_GCP_Customers= '''

select *
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
CROSS JOIN UNNEST(list_tpl_pi_alias_str) as param
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and pi_prod_instnc_stat_cd in ('C') #Canceled service
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and DATE(pi_prod_instnc_stat_ts)>='2022-08-01' and DATE(pi_prod_instnc_stat_ts)<'2022-09-01' #Date on which the status was updated


'''